### Modelos
- Modelo Óbito-Curado (60/40): Regressão
- Modelo Internado-Curado (70/30): AdaBoost

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import import_ipynb
import modelofuncoes as mf

from joblib import dump, load

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score

importing Jupyter notebook from modelofuncoes.ipynb


### Curva ROC

In [ ]:
def exibirCurvaROC(modeloClassificador, x_test, y_test):
    y_preds = modeloClassificador.predict_proba(x_test)

    # take the second column because the classifier outputs scores for the 0 class as well
    preds = y_preds[:,1]

    # fpr (false-positive-rate) e tpr (true-positive-rate)
    fpr, tpr, _ = metrics.roc_curve(y_test, preds)

    auc_score = metrics.auc(fpr, tpr)

    # Construindo o Gráfico
    plt.clf()
    plt.title('ROC Curve')
    plt.plot(fpr, tpr, label='AUC = {:.2f}'.format(auc_score))
    plt.plot([0,1],[0,1],'r--')

    plt.xlim([-0.1,1.1])
    plt.ylim([-0.1,1.1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')

    plt.legend(loc='lower right')
    plt.show()

### Avaliando o Smote

In [ ]:
from imblearn.over_sampling import RandomOverSampler, SMOTE

def aplicarSmote(base, alvo):

    X_smt = base.drop(alvo, axis = 1)
    y_smt = base[alvo]
    
    # Contar o número de ocorrências
    # np.bincount(y_smt)
    
    # Carregar o Smote
    smt = SMOTE()
    
    # Aplicar o Smote
    X_smt, y_smt = smt.fit_resample(X_smt, y_smt)
    
    # Contar o número de ocorrências
    # np.bincount(y_smt)
    
    # Convertendo em DF
    smoteBase = pd.DataFrame(X_smt)
    smoteBase[alvo] = y_smt
    
    return smoteBase

In [ ]:
hiperparametrosGB = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, 
                           random_state=1859, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

## Desempenho dos Modelos Escolhidos
---------

### Definindo as bases de dados

In [11]:
# Modelo OC (7030 e Smote 7030)
# url = 'PreProcessamento/balanceadas/OC/ba-oc-7030-sp.xlsx',
url = 'PreProcessamento/balanceadas/Smote/sm-oc-7030-sp.xlsx'

# Modelo IC (Smote 6040)
# url = 'PreProcessamento/balanceadas/Smote/sm-ic-6040-sp.xlsx'

alvo = 'evolucaoCaso'
baseFinal = pd.read_excel(url)

In [12]:
baseFinal.shape()

(6678, 38)

### Reduzindo os Atributos

In [13]:
# (Modelo OC)
# atributosSelecionados = ['faixaetaria', 'dispneia', 'qntVacinas', 'coriza', 'dorDeGarganta', 
#                        dorDeCabeca', 'obesidade', 'diabetes', 'faixaDiasSintomas', 'evolucaoCaso']

# (Modelo IC)
# atributosSelecionados = ['faixaetaria', 'dorDeCabeca', 'dorDeGarganta', ‘renal’, 'dispneia', 'diabetes',  
#                        ‘tosse’, 'qntVacinas', 'faixaDiasSintomas', 'coriza', ‘febre’, evolucaoCaso']

baseFinalReduzida = baseFinal.loc[:, atributosSelecionados]

NameError: name 'atributosSelecionados' is not defined

### Carregando o Modelo Escolhido

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(baseFinalReduzida.drop(alvo,axis=1),
                                                    baseFinalReduzida[alvo],
                                                    test_size=0.3,
                                                    random_state=42);

In [ ]:
# Fit nos dados
modeloClassificador = hiperparametrosGB
modeloClassificador.fit(x_train, y_train)

#Realizando a predição
resultadoTeste = modeloClassificador.predict(x_test)

# Verificando a acurácia
print(metrics.classification_report(y_test, resultadoTeste))

In [ ]:
exibirCurvaROC(modeloClassificador, x_test, y_test)

------

### Selecionando as bases para a Validação

In [16]:
# Bases - OC
top10_bases_modelo_oc_map = {'MG': 'PreProcessamento/balanceadas/OC/ba-oc-7030-mg.xlsx',
                         'BA': 'PreProcessamento/balanceadas/OC/ba-oc-7030-ba.xlsx',
                         'PA': 'PreProcessamento/balanceadas/OC/ba-oc-7030-pa.xlsx',
                         'RO': 'PreProcessamento/balanceadas/OC/ba-oc-7030-ro.xlsx',
                         'RJ': 'PreProcessamento/balanceadas/OC/ba-oc-7030-rj.xlsx',
                         'SC': 'PreProcessamento/balanceadas/OC/ba-oc-7030-sc.xlsx',
                         'CE': 'PreProcessamento/balanceadas/OC/ba-oc-7030-ce.xlsx',
                         'RS': 'PreProcessamento/balanceadas/OC/ba-oc-7030-rs.xlsx',
                         'TO': 'PreProcessamento/balanceadas/OC/ba-oc-7030-to.xlsx',
                         'MT': 'PreProcessamento/balanceadas/OC/ba-oc-7030-mt.xlsx',
                        }

default_path = 'PreProcessamento/balanceadas/OC/'
top10_bases_modelo_oc = ['ba-oc-7030-mg.xlsx', 'ba-oc-7030-ba.xlsx', 'ba-oc-7030-pa.xlsx', 
                         'ba-oc-7030-ro.xlsx', 'ba-oc-7030-rj.xlsx', 'ba-oc-7030-sc.xlsx', 
                         'ba-oc-7030-ce.xlsx', 'ba-oc-7030-rs.xlsx', 'ba-oc-7030-to.xlsx', 
                         'ba-oc-7030-mt.xlsx']

In [ ]:
# Bases - IC
top10_bases_modelo_ic = {'BA': 'PreProcessamento/balanceadas/IC/ba-ic-6040-sp.xlsx',
                        'MG': 'PreProcessamento/balanceadas/IC/ba-ic-6040-mg.xlsx',
                        'SC': 'PreProcessamento/balanceadas/IC/ba-ic-6040-sc.xlsx',
                        'RS': 'PreProcessamento/balanceadas/IC/ba-ic-6040-rs.xlsx',
                        'RJ': 'PreProcessamento/balanceadas/IC/ba-ic-6040-rj.xlsx',
                        'PB': 'PreProcessamento/balanceadas/IC/ba-ic-6040-pb.xlsx',
                        'GO': 'PreProcessamento/balanceadas/IC/ba-ic-6040-go.xlsx',
                        'CE': 'PreProcessamento/balanceadas/IC/ba-ic-6040-ce.xlsx',
                        'DF': 'PreProcessamento/balanceadas/IC/ba-ic-6040-df.xlsx',
                        'PA': 'PreProcessamento/balanceadas/IC/ba-ic-6040-pa.xlsx'
                        }

In [18]:
top10_bases_modelo_oc.values()

dict_values(['PreProcessamento/balanceadas/OC/ba-oc-7030-mg.xlsx', 'PreProcessamento/balanceadas/OC/ba-oc-7030-ba.xlsx', 'PreProcessamento/balanceadas/OC/ba-oc-7030-pa.xlsx', 'PreProcessamento/balanceadas/OC/ba-oc-7030-ro.xlsx', 'PreProcessamento/balanceadas/OC/ba-oc-7030-rj.xlsx', 'PreProcessamento/balanceadas/OC/ba-oc-7030-sc.xlsx', 'PreProcessamento/balanceadas/OC/ba-oc-7030-ce.xlsx', 'PreProcessamento/balanceadas/OC/ba-oc-7030-rs.xlsx', 'PreProcessamento/balanceadas/OC/ba-oc-7030-to.xlsx', 'PreProcessamento/balanceadas/OC/ba-oc-7030-mt.xlsx'])

### Executando a Validação

In [ ]:
alvos = [0, 1]
desempenhoDosModelos = pd.DataFrame()
for indice in range(0, len(listaBases)):
    covidData = pd.read_excel(listaBases[indice])
    dadosTeste = covidData.loc[:, colunasSelecionadasTeste]
    dadosAlvo =  covidData.loc[:, alvo]
     
    metricasMap = calcularMetricas(dadosTeste, dadosAlvo, listaBases[indice], modeloClassificador, alvos)
    desempenhoDosModelos = pd.concat([desempenhoDosModelos, pd.DataFrame([metricasMap])]) 

### Calculando a média das Métricas

In [14]:
# Verificando o Desempenho do Modelo
# desempenhoDosModelos = mf.testarModelos(bases, alvo, colunasRemovidas, modeloClassificador, valoresAlvo)
# desempenhoDosModelos

In [15]:
# Identificando a média 
# media = desempenhoDosModelos.mean()

# Inserindo as outras colunas
# media['Base'] = 'Média dos Modelos'
# media['Modelo'] = "-"

# Adicionando a média dos valores
# desempenhoDosModelos = desempenhoDosModelos.append(media, ignore_index=True)
# desempenhoDosModelos

# Convertendo series para dict
# media_dict = pd.Series(media.values, index=media.keys).to_dict()

### Definindo a Função da Curva ROC

### Funções Utilizadas

In [ ]:
def __transformarEmPorcentagem__(valor):
    porcentagem = str(round(valor * 100, 3)) + "%"
    return porcentagem

In [ ]:
def calcularMetricas(x_test, y_test, baseDeDados, classificador, alvos):
    metricas = {}
    
    #Realizando a predição
    resultadoPredicao = classificador.predict(x_test)

    # Verificando a acurácia
    metricas['Acurácia'] = __transformarEmPorcentagem__(accuracy_score(y_test, resultadoPredicao))
    
    # Criando um dicionaŕio a partir das métricas
    metricasValores = metrics.classification_report(y_test, resultadoPredicao, output_dict=True)
    df = pd.DataFrame(metricasValores).transpose()
    
    metricas["Base"] = baseDeDados

    for classe in alvos:
        metricas["Precisão (" + str(classe) + ")"] = __transformarEmPorcentagem__(df.loc[str(classe)]['precision'])
        metricas["Recall (" + str(classe) + ")"] = __transformarEmPorcentagem__(df.loc[str(classe)]['recall'])
        metricas["F1-Score (" + str(classe) + ")"] = __transformarEmPorcentagem__(df.loc[str(classe)]['f1-score'])
    
    return metricas

### Modelo OC - Avaliando as outras Bases (Estados)

In [ ]:
listaBases = ["Vacinas/OC/oc-vacina-6040-ac.xlsx", "Vacinas/OC/oc-vacina-6040-al.xlsx", "Vacinas/OC/oc-vacina-6040-am.xlsx", "Vacinas/OC/oc-vacina-6040-ap.xlsx", "Vacinas/OC/oc-vacina-6040-ba.xlsx", 
              "Vacinas/OC/oc-vacina-6040-ce.xlsx", "Vacinas/OC/oc-vacina-6040-df.xlsx", "Vacinas/OC/oc-vacina-6040-go.xlsx", "Vacinas/OC/oc-vacina-6040-ma.xlsx", "Vacinas/OC/oc-vacina-6040-mg.xlsx", 
              "Vacinas/OC/oc-vacina-6040-mt.xlsx", "Vacinas/OC/oc-vacina-6040-pa.xlsx", "Vacinas/OC/oc-vacina-6040-pb.xlsx", "Vacinas/OC/oc-vacina-6040-pe.xlsx", "Vacinas/OC/oc-vacina-6040-pi.xlsx", 
              "Vacinas/OC/oc-vacina-6040-rj.xlsx", "Vacinas/OC/oc-vacina-6040-rn.xlsx", "Vacinas/OC/oc-vacina-6040-ro.xlsx", "Vacinas/OC/oc-vacina-6040-rr.xlsx", "Vacinas/OC/oc-vacina-6040-rs.xlsx"] 

alvo = 'evolucaoCaso'
colunasSelecionadasTeste = ['faixaetaria', 'dispneia', 'qntVacinas', 'dorDeGarganta', 
                            'coriza', 'diabetes', 'dorDeCabeca', 'cardiaca']

In [ ]:
alvos = [0, 1]
desempenhoDosModelos = pd.DataFrame()
for indice in range(0, len(listaBases)):
    covidData = pd.read_excel(listaBases[indice])
    dadosTeste = covidData.loc[:, colunasSelecionadasTeste]
    dadosAlvo =  covidData.loc[:, alvo]
     
    metricasMap = calcularMetricas(dadosTeste, dadosAlvo, listaBases[indice], modeloClassificador, alvos)
    desempenhoDosModelos = pd.concat([desempenhoDosModelos, pd.DataFrame([metricasMap])]) 

### Modelo IC - Analisando outras Regiões

In [ ]:
# Lista das Bases de Regiões
listaBases = ["Vacinas/Regioes/IC/ic-vacina-6040-centrooeste.xlsx", "Vacinas/Regioes/IC/ic-vacina-6040-sudeste.xlsx", 
               "Vacinas/Regioes/IC/ic-vacina-6040-norte.xlsx", "Vacinas/Regioes/IC/ic-vacina-6040-nordeste.xlsx"]

# Colunas Utilizadas
colunasSelecionadasTeste = ['faixaetaria', 'sexo', 'racaCor', 'dorDeGarganta', 'dispneia', 'coriza', 
                            'dorDeCabeca', 'diabetes', 'renal', 'imunossupressao', 'fragilidadeImuno', 
                            'gestante', 'cardiaca', 'qntVacinas']

alvo = 'evolucaoCaso'

In [ ]:
alvos = [1, 2]
desempenhoDosModelos = pd.DataFrame()
for indice in range(0, len(listaBases)):
    covidData = pd.read_excel(listaBases[indice])
    dadosTeste = covidData.loc[:, colunasSelecionadasTeste]
    dadosAlvo =  covidData.loc[:, alvo]
     
    metricasMap = calcularMetricas(dadosTeste, dadosAlvo, listaBases[indice], modeloClassificador, alvos)
    desempenhoDosModelos = pd.concat([desempenhoDosModelos, pd.DataFrame([metricasMap])])
desempenhoDosModelos